In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from classes import WikiArts

from torch.utils.data import DataLoader


In [2]:
path_labels = r'./output/path_label.csv'
path_images = r'./wikiart_500_paintings'

# Image pre-processing

### Transform images (normalization, resize, to tensor)

In [3]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

transform = transforms.Compose([transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        
data = WikiArts(path_labels, path_images, transform)


### Split data

In [4]:
train_dataset, test_dataset = torch.utils.data.random_split(data, [len(data) - 779, 779])
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [len(train_dataset) - 779, 779])

### Data loaders to load data in batches

In [5]:
num_workers = 4
batch_size = 25

In [6]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True, num_workers=num_workers)
valid_loader = DataLoader(dataset = valid_dataset, batch_size = batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True, num_workers=num_workers)

/storage/home/hpaceice1/hstrauss6/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Import pre-trained model

In [7]:
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)

Using cache found in /storage/home/hpaceice1/hstrauss6/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/storage/home/hpaceice1/hstrauss6/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:14: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"
/storage/home/hpaceice1/hstrauss6/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:18: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"


In [8]:
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layers): Sequential(
    (0): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d

In [9]:
# Replace the last fully connected layer by a new one with the number of artists we want to classify. In our example case, let us take 2

num_classes = 28 #TO CHANGE REGARDING THE NUMBER OF ARTISTS WE WANT
resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, num_classes)

In [10]:
resnet50.fc

Linear(in_features=2048, out_features=28, bias=True)

# Train the model

In [11]:
device_string = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_string)
resnet50 = resnet50.to(device)
device_string

'cpu'

In [12]:
num_epochs = 7
learning_rate = 0.001
mom = 0.9

#### Optimizer: SGD
#### Loss function: cross entropy

In [13]:
optimizer = optim.SGD(resnet50.parameters(), lr=learning_rate, momentum=mom)

In [ ]:
BEST_MODEL_PATH = 'best_model_optmiSGD_lossCrossEntrop.pth'
BEST_OPTIMIZER_PATH = 'best_optim_optmiSGD_lossCrossEntrop.pth'
best_accuracy = 0.0
train_losses = []
valid_losses = []
valid_accuracies = []

for epoch in range(num_epochs):
    train_loss = 0.0
    valid_loss = 0.0

    # Train set
    resnet50.train()
    # i = 0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = resnet50(images)
        labels=torch.from_numpy(
            np.array([labels[i] for i in range (len(labels))])).long()
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    # Valid set
    resnet50.eval()
    valid_error_count = 0.0
    for data, target in valid_loader:     
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        loss = F.cross_entropy(outputs, labels)
        valid_loss += loss.item() * images.size(0)
        valid_error_count += float(len(labels[labels != outputs.argmax(1)]))

    # Comparison valid and test set for each epoch
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    # Calculate accuracy for validation dataset
    validation_accuracy = 1.0 - float(valid_error_count) / float(len(valid_dataset))
    valid_accuracies.append(validation_accuracy)
    print('%d: %f' % (epoch, validation_accuracy))
    if validation_accuracy >= best_accuracy:
        torch.save(resnet50.state_dict(), BEST_MODEL_PATH)
        torch.save(optimizer.state_dict(), BEST_OPTIMIZER_PATH)
        best_accuracy = validation_accuracy

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)

In [ ]:
# Confirm results on test set - with last model
    # Test set
test_accuracies = []

resnet50.eval()
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        outputs = resnet50(images)
        for x in outputs:
            probabilities = torch.nn.functional.softmax(x, dim=0)
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    test_accuracies.append(test_accuracy)
    print('accuracy test set: %f' % (test_accuracy))


In [ ]:
# Confirm results on test set - with first model
path = 'best_model_optmiSGD_lossCrossEntrop.pth'
path_optim = 'best_optim_optmiSGD_lossCrossEntrop.pth'

models = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes = 28 #TO CHANGE REGARDING THE NUMBER OF ARTISTS WE WANT
models.fc = torch.nn.Linear(models.fc.in_features, num_classes)

optimizers = optim.SGD(models.parameters(), lr=0.001, momentum=0.9)

models.load_state_dict(torch.load(path))

models.eval()
with torch.no_grad(): 
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        outputs = models(images)
        for x in outputs:
            probabilities = torch.nn.functional.softmax(x, dim=0)
        test_error_count += float(len(labels[labels != outputs.argmax(1)]))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    test_accuracies.append(test_accuracy)
    print('accuracy test set: %f' % (test_accuracy))



#### Optimizer: AdaGrad
#### Loss function: cross entropy

In [ ]:
resnet50_2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
num_classes = 28 #TO CHANGE REGARDING THE NUMBER OF ARTISTS WE WANT
resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, num_classes)
num_epochs = 7
learning_rate = 0.001
mom = 0.9

In [ ]:
optimizer_2 = optim.Adagrad(resnet50_2.parameters(), lr=learning_rate)

In [ ]:
BEST_MODEL_PATH_2 = 'best_model_optmiAdaGrad_lossCrossEntrop.pth'
BEST_OPTIMIZER_PATH_2 = 'best_optim_optmiAdaGrad_lossCrossEntrop.pth'
best_accuracy_2 = 0.0
train_losses_2 = []
valid_losses_2 = []
valid_accuracies_2 = []

for epoch in range(num_epochs):
    train_loss_2 = 0.0
    valid_loss_2= 0.0

    # Train set
    resnet50_2.train()
    # i = 0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = resnet50_2(images)
        labels=torch.from_numpy(
            np.array([labels[i] for i in range (len(labels))])).long()
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss_2 += loss.item() * images.size(0)
    
    # Valid set
    resnet50_2.eval()
    valid_error_count_2 = 0.0
    for data, target in valid_loader:     
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50_2(images)
        loss = F.cross_entropy(outputs, labels)
        valid_loss_2 += loss.item() * images.size(0)
        valid_error_count_2 += float(len(labels[labels != outputs.argmax(1)]))

    # Comparison valid and test set for each epoch
    train_loss_2 = train_loss_2/len(train_loader.sampler)
    valid_loss_2 = valid_loss_2/len(valid_loader.sampler)
    train_losses_2.append(train_loss_2)
    valid_losses_2.append(valid_loss_2)

    # Calculate accuracy for validation dataset
    validation_accuracy_2 = 1.0 - float(valid_error_count_2) / float(len(valid_dataset))
    valid_accuracies_2.append(validation_accuracy_2)
    print('%d: %f' % (epoch, validation_accuracy_2))
    if validation_accuracy_2 >= best_accuracy_2:
        torch.save(resnet50_2.state_dict(), BEST_MODEL_PATH_2)
        torch.save(optimizer_2.state_dict(), BEST_OPTIMIZER_PATH_2)
        best_accuracy_2 = validation_accuracy_2